# Анализ Университетов Мира (2023)
Я всех приветствую в очередной работе, где мы будет на примере датасета, содержащим данные по многим университетам мира, рассматривать университеты России и мира в целом, чтобы понять, куда,
возможно, стоит пойти учиться, если вдруг вы захотите целенаправленно изучать что-то для себя новое в разных странах и сферах в целом.

Данные я взял уже из готовой таблицы, собранно во время анализа в 2023 году, с сайта Kaggle.com.
Посмотреть и скачать данныые можно по ссылке: https://www.kaggle.com/datasets/tariqbashir/world-university-ranking-2023

---
## Описание датасета
```
World University Rankings 2023 is based upon 1,799 universities across 104 countries and regions based on many (at least 13) performance indicators that measure teaching, research, knowledge transfer, and international outlook. Data was collected from over 2,500 institutions, including survey responses from 40,000 scholars and analysis of over 121 million citations in 15.5 million research publications. The US has the most institutions overall and in the top 200, but China has overtaken Australia for the fourth-highest number of institutions in the top 200. The University of Oxford is ranked first for this year, while the highest new entry is Italy's Humanitas University.
```

---

Комментарий: Судя по данным, даже после обработки всех значений и к приведению их к нормальному виду получается >2000 (2345 rows если быть точным) университетов, поэтому фраза `based upon 1,799 universities` заставляет меня задуматься, где это я ошибаюсь в вычислениях
## Импорт необходимых библиотек Python


In [198]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [199]:
db=pd.read_csv('dataset.csv',encoding='ISO-8859-1') # Без данного encoding параметра вылезет ошибка чтения файла
db

,Rank,Name,No. of FTE Students,No. of students per staff,International Students,Female:Male Ratio
0,NaN,Country/Region,NaN,NaN,NaN,NaN
1,Reporter,Zarqa University,"5,768",18.1,32%,47:53:00
2,NaN,Jordan,NaN,NaN,NaN,NaN
3,Reporter,Ziauddin University,"4,906",8.8,1%,63:37:00
4,NaN,Pakistan,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
4872,NaN,United Kingdom,NaN,NaN,NaN,NaN
4873,2,Harvard University,"21,887",9.6,25%,50:50:00
4874,NaN,United States,NaN,NaN,NaN,NaN
4875,1,University of Oxford,"20,965",10.6,42%,48:52:00


## Комментарий к датасету
Вы только посмотрите на это безумие! NaN Строки, в одном столбце находятся разные параметры, нужно срочно приводить всё к нормальному виду.

---
## Нормализация таблицы

In [200]:
db.drop(labels=0, inplace=True)
db.reset_index(inplace=True, drop=True)
def find_irregularity(db,full=False):
    ''' 
    Проверим, вся ли таблица имеет такой вид, или она неоднородна
    '''
    prev=-1
    sum=0
    for index, row in db.iterrows():
        # print(index,row['Rank'])
        if pd.isna(row['Rank']):
            if index-prev!=2:
                sum+=1
                if sum<3 or full==True: #Чтобы вывод не забивался поставил ограничение на вывод: sum<10
                    print('-------------------------')
                    print(f'Внимание, появилась неоднородность в данных')
                    for i in range(index-2,index+3):
                        print(i,db['Rank'].iloc[i],db['Name'].iloc[i], index-prev)
            prev=index
    print(f"---------------------\nКол-во неодноровностей:{sum}")
    if sum==0:
        print(f"Congrats, all clear!")
find_irregularity(db)

-------------------------
Внимание, появилась неоднородность в данных
38 Reporter Western Caspian University 1
39 nan Azerbaijan 1
40 nan Explore 1
41 Reporter ?Wellspring University 1
42 nan Nigeria 1
-------------------------
Внимание, появилась неоднородность в данных
67 Reporter National University of Uzbekistan named after Mirzo Ulugbek 1
68 nan Uzbekistan 1
69 nan Explore 1
70 Reporter University of Uyo 1
71 nan Nigeria 1
---------------------
Кол-во неодноровностей:186


## Вывод:
Как видите, тут есть неоднородность в данных. Помимо того, что в одном столбце находятся и название университетов, и их страны. Так еще есть некая строка Explore, которая меняет последовательность, не давая мне правильно и быстро переделать таблицу, не портя ее структуру.
Если внимательно изучить таблицу, то можно предположить, что строки Explore нам вообще не нужны, они ничего нам не дают от слова совсем. Уберем их отсюда.

In [201]:
db=db[db['Name']!='Explore']
db.reset_index(inplace=True, drop=True)
find_irregularity(db, full=True)

-------------------------
Внимание, появилась неоднородность в данных
2346 10011200 St Marianna University School of Medicine 1
2347 nan Japan 1
2348 nan Not accredited 1
2349 10011200 ?tefan cel Mare University of Suceava 1
2350 nan Romania 1
---------------------
Кол-во неодноровностей:1


Как видите, осталась только одна неоднородность, причем очень странная. Посмотрим на нее поближе

In [202]:
db.iloc[2345:2351]

,Rank,Name,No. of FTE Students,No. of students per staff,International Students,Female:Male Ratio
2345,NaN,United Kingdom,NaN,NaN,NaN,NaN
2346,10011200,St Marianna University School of Medicine,833,0.9,1%,39 : 61
2347,NaN,Japan,NaN,NaN,NaN,NaN
2348,NaN,Not accredited,NaN,NaN,NaN,NaN
2349,10011200,?tefan cel Mare University of Suceava,"9,239",24.8,14%,59:41:00
2350,NaN,Romania,NaN,NaN,NaN,NaN


Как видно, есть по сути пустая строка с ненаписанными данными тк университет, скорее всего, Не аккредитован.
> Unaccredited Universities is a list of colleges, universities, and other institutions that do not have the equivalent of regional academic accreditation. Some of these institutions may have legal authority to enroll students and grant degrees, but do not have regional academic accreditation for various reasons.

Найти не аккредитованные университеты можно по ссылке: https://www.scholaro.com/unaccredited-universities/

В нашем случае, просто избавимся от всех таких строк, если они есть (вроде одна только)

In [203]:
db=db[db['Name']!='Not accredited']
db.reset_index(inplace=True, drop=True)
find_irregularity(db, full=True) 

---------------------
Кол-во неодноровностей:0
Congrats, all clear!


In [204]:
countries=[]
indexes=[]
for index, row in db.iterrows():
    if index%2==1:
        countries.append(row['Name'])
        indexes.append(index)
db.drop(labels=indexes, inplace=True)
db.reset_index(inplace=True, drop=True)

In [205]:
db.insert(2,'Country',countries)
db

,Rank,Name,Country,No. of FTE Students,No. of students per staff,International Students,Female:Male Ratio
0,Reporter,Zarqa University,Jordan,"5,768",18.1,32%,47:53:00
1,Reporter,Ziauddin University,Pakistan,"4,906",8.8,1%,63:37:00
2,Reporter,Zhytomyr Polytechnic State University,Ukraine,"3,869",15.4,1%,34 : 66
3,Reporter,"Yusuf Maitama Sule University, Kano",Nigeria,"12,880",33.0,0%,48:52:00
4,Reporter,York St John University,United Kingdom,"6,315",18.6,12%,65:35:00
...,...,...,...,...,...,...,...
2340,5,Massachusetts Institute of Technology,United States,"11,415",8.2,33%,40 : 60
2341,3,Stanford University,United States,"16,164",7.1,24%,46:54:00
2342,3,University of Cambridge,United Kingdom,"20,185",11.3,39%,47:53:00
2343,2,Harvard University,United States,"21,887",9.6,25%,50:50:00


---
## Ошибки в тексте

Вы могли уже заметить, что некоторые значения выглядят немного странно. Давайте я вам перечислю, что нас не может устраивать в данных:

1. Данные из столбцов, к примеру No. of FTE Students написаны с запятой а не с точкой, поэтому числа не воспринимаются как числа. Давайте убедимся в этом, проверив тип данных у всех значений.

In [206]:
db.dtypes

Rank                          object
Name                          object
Country                       object
No. of FTE Students           object
No. of students per staff    float64
International Students        object
Female:Male Ratio             object
dtype: object

Как видите, только у столбца 'No. of students per staff' нет проблем (пока что). Давайте менять данные, приводя их к нормальному виду!

p.s. На самом деле, стоило бы руки оторвать тому человеку, который эти данные в таком виде собрал. Уж прогнать их через Power Query можно было бы, чтобы не было у нас уже проблем с обработкой.



In [207]:
for index, row in db.iterrows():
    db['No. of FTE Students'][index]=db['No. of FTE Students'][index].replace(',', '.')
    db['No. of FTE Students'][index]=float(db['No. of FTE Students'][index])

print(db['No. of FTE Students'])

C:\Users\enoki\AppData\Local\Temp\ipykernel_7600\2364009046.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['No. of FTE Students'][index]=db['No. of FTE Students'][index].replace(',', '.')
C:\Users\enoki\AppData\Local\Temp\ipykernel_7600\2364009046.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['No. of FTE Students'][index]=float(db['No. of FTE Students'][index])


0        5.768
1        4.906
2        3.869
3        12.88
4        6.315
         ...  
2340    11.415
2341    16.164
2342    20.185
2343    21.887
2344    20.965
Name: No. of FTE Students, Length: 2345, dtype: object
